In [2]:
#using pandas read_json command with orient='records' option to display the dataframe
import pandas as pd
df1=pd.read_json('file.json', orient='records')

In [106]:
#time is represented in unix epoch time format in the dataset and so using unit='s' in pd.to_datetime to tranform date-time
df1['approx_payout_date'] =  pd.to_datetime(df1['approx_payout_date'], unit='s')

In [107]:
df1['approx_payout_date'].head()

0   2010-02-13 12:00:00
1   2011-02-03 08:00:00
2   2011-01-28 00:00:00
3   2014-01-06 00:00:00
4   2011-02-17 00:00:00
Name: approx_payout_date, dtype: datetime64[ns]

In [109]:
df1['event_created'] =  pd.to_datetime(df1['event_created'], unit='s')
df1['event_end'] =  pd.to_datetime(df1['event_end'], unit='s')
df1['event_start'] =  pd.to_datetime(df1['event_start'], unit='s')
df1['user_created'] =  pd.to_datetime(df1['user_created'], unit='s')

In [136]:
df1['acct_type'].unique()

array(['fraudster_event', 'premium', 'spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock',
       'tos_warn', 'fraudster_att', 'spammer_web', 'spammer'],
      dtype=object)

In [15]:
#coulmns of the dataframe
df1.columns

Index(['acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'description', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type', 'previous_payouts', 'sale_duration',
       'sale_duration2', 'show_map', 'ticket_types', 'user_age',
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state'],
      dtype='object')

In [ ]:
#what i think could be useful features:
'acct_type', 'approx_payout_date','country','currency''email_domain','sale_duration''sale_duration2'
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published''num_order', 'num_payouts''org_facebook', 'org_name', 'org_twitter''user_type''venue_name', 'venue_state'

In [104]:
#obviously need more details about the delivery_method
df1['delivery_method'].unique()

array([ 0.,  1.,  3., nan])

In [105]:
df1['email_domain'].unique()

array(['gmail.com', 'ruf.org', 'pvsd.k12.ca.us', ..., 'abdcycling.com',
       'newcastle-cu.com', 'sbm.ie'], dtype=object)

In [106]:
#could visualize using user_type. if a user_type entails both fraud and non-fraud transactions then it might be a good idea to 
#revise the categories for more clarity!

df1['user_type'].unique()

array([  1,   3,   4,   5, 103,   2])

In [111]:
#expanding the lists of dictionaries in df1['ticket_types'] to a dataframe
res=[]
for i in df1['ticket_types']:
    for j in i:
        res.append(j)
df_ticket = pd.DataFrame.from_dict(res)

In [113]:
#expanding the lists of dictionaries in df1['previous_payouts'] to a dataframe but this leads to a dataframe of length:1097472
#
ans=[]
for i in df1['previous_payouts']:
    for j in i:
        ans.append(j)
df_payouts = pd.DataFrame.from_dict(ans)

In [115]:
len(df_payouts)

1097472

In [90]:
#this list of countries is from the list of dictionaries of df1['previous_payouts']
df_payouts['country'].unique()

array(['US', 'IE', '', 'FR', 'CA', 'GB', 'AU', 'ES', 'BE', 'NL', 'DE',
       'NZ', 'United Kingdom', 'Australia', 'AT', 'Canada', 'AUSTRALIA',
       'UNITED KINGDOM', 'Eventbrite', 'EVENTBRITE', 'EVENT', 'ICELAND',
       'UK', 'France', 'England', 'Indonesia', 'New Zealand', 'Germany',
       'Thailand'], dtype=object)

In [116]:
#this list of countries is from the dataset itself - interesting could look for an option to join?

#eureka: why not count the number of countries (each country) from df1['previous_payouts'] - and create that many empty rows
#in the dataset and then merge ? should check whether the same method would work for df1['ticket_types'] too?! but this is great - ideas are always great - happy!
df1['country'].unique()

array(['US', 'IE', 'FR', 'CA', 'GB', '', 'AU', 'ES', 'NL', 'DE', 'VN',
       'MY', 'NZ', 'PK', 'MA', 'AR', 'MX', 'CH', None, 'SG', 'TH', 'BE',
       'PH', 'A1', 'CI', 'AT', 'ID', 'PS', 'PT', 'TR', 'NI', 'KE', 'IT',
       'HU', 'RS', 'RO', 'NG', 'CZ', 'PR', 'AE', 'BS', 'KH', 'JM', 'IN',
       'NA', 'FI', 'HR', 'BG', 'VI', 'TJ', 'GH', 'PE', 'QA', 'SI', 'GR',
       'BB', 'CM', 'IS', 'SE', 'RU', 'DZ', 'VE', 'UY', 'ZA', 'IM', 'LB',
       'CR', 'IL', 'CN', 'DK', 'CO', 'EC', 'JE'], dtype=object)

In [117]:
len(df1.groupby(['country']))

72

In [98]:
df_pay_new=df_payouts.groupby(['country'])
len(df_pay_new)

29

In [118]:
#Keep only date part when using pandas.to_datetime

df1['approx_payout_date']=df1['approx_payout_date'].dt.date


In [119]:
df1['event_created']=df1['event_created'].dt.date
df1['event_end']=df1['event_end'].dt.date
df1['event_start'] = df1['event_start'].dt.date
df1['user_created'] = df1['user_created'].dt.date

In [26]:
#going to convert the acc_type column values based on the values in the same column. for example: if 'fraud' appears in the column then
#say True for being a fraud transaction or say False for being not a fraud transaction
#first check the unique values of the column: acct_type
df1['acct_type'].unique()


array(['fraudster_event', 'premium', 'spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock',
       'tos_warn', 'fraudster_att', 'spammer_web', 'spammer'],
      dtype=object)

In [121]:
#to convert the values of the column: acct_type
#if value: 'fraudster' appear then 1 as value or 0

df1['acct_type'] = df1['acct_type'].replace(['fraudster_event', 'fraudster', 'fraudster'], 1)

df1['acct_type']=df1['acct_type'].replace(['premium', 'spammer_warn','spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock','tos_warn','spammer_web', 'spammer'], 0)


In [123]:
df1['acct_type']

0        1
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       1
27       0
28       0
29       0
        ..
14307    0
14308    0
14309    0
14310    0
14311    0
14312    0
14313    0
14314    0
14315    0
14316    1
14317    0
14318    1
14319    0
14320    0
14321    0
14322    0
14323    0
14324    0
14325    1
14326    0
14327    0
14328    0
14329    0
14330    0
14331    0
14332    1
14333    0
14334    0
14335    0
14336    1
Name: acct_type, Length: 14337, dtype: object